# Vector/Pipelining scenario #1: YUV to RGB conversion

Example YUV to RGB conversion (simplified, not clamping)

**Note** Not verified for correct conversion, concept study only

Makes use of the `VectorSig` data type and the simple `@pipeline` decorator.

In [1]:
!pip install numpy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [2]:
import sys
sys.path.insert(0, '../..')

Import video types:

In [3]:
from video.color import *
from video.videotypes import *

Import pipeline and target auxiliaries:

In [4]:
from myirl.library.pipeline import *
from myirl import targets

Construct the conversion matrix, in this case for JPEG-compliant YCrCb:

In [5]:
CLAMP = False
LEVELSHIFT = False
BPP        = 8
FRACT_SIZE = 16
CALCSIZE   = FRACT_SIZE + BPP
SATURATION_VALUE_MAX = 127 # YUV maximum value (saturation)
SATURATION_VALUE_MIN = -128 # YUV minimum value (saturation)


# Signed matrix entries:
Y_FROM_RGB = vector_to_fp(FRACT_SIZE, 1, mat_jpeg_rgb2yuv[0])
U_FROM_RGB = vector_to_fp(FRACT_SIZE, 1, mat_jpeg_rgb2yuv[1])
V_FROM_RGB = vector_to_fp(FRACT_SIZE, 1, mat_jpeg_rgb2yuv[2])

def F(x, s = FRACT_SIZE):
    return intbv(x)[s:]

YUV_SLICE = slice(CALCSIZE-1, CALCSIZE-1 - BPP)

MATRIX = [
    [ F(Y_FROM_RGB[i]) for i in range(3) ],
    [ F(U_FROM_RGB[i]) for i in range(3) ],
    [ F(V_FROM_RGB[i]) for i in range(3) ]
]

from myirl.vector import VectorSignal

I = lambda x: ( x[i]._val for i in range(3) )

# @bulkwrapper()
# class RGBParam:
#     def __init__(self):
#         self.y = VectorSig(3, MATRIX[0], initializer = I(MATRIX[0]))
#         self.u = VectorSig(3, MATRIX[1], initializer = I(MATRIX[1]))
#         self.v = VectorSig(3, MATRIX[1], initializer = I(MATRIX[2]))

In [6]:
MATRIX

[[intbv(9797), intbv(19234), intbv(3735)],
 [intbv(60007), intbv(54682), intbv(16384)],
 [intbv(16384), intbv(51817), intbv(62872)]]

In [7]:
from myirl import simulation as sim
from myirl.test.common_test import gen_osc

@block
def video_rgb_yuv(clk : ClkSignal,
                  vin : VideoPort,
                  rgb : Signal,
                  param_matrix : list,
                  vout : VideoPort.Output,
                  yuv : Signal.Output,):
    """RGB to full range YUV422 converter, manual pipeline inference"""
    
    py, pu, pv = [
        VectorSignal(3, F(0), initializer = I(param_matrix[i]), name = "p_coef%d" % i)  \
        for i in range(3)
    ]
    
    # Use initializers:
    py._init = True
    pu._init = True
    pv._init = True
    
    valid = Signal(bool())
    
    rgb_v = VectorSignal(3, FractUnsigned(0, BPP), name = 'rgbv')
    
    a = VectorSignal(3, FractSigned(0, CALCSIZE+2), name = "add_res")
    y = VectorSignal(3, FractUnsigned(0, CALCSIZE), name = "ydata")
    u, v = [ VectorSignal(3, FractSigned(0, CALCSIZE+1), name = n) for n in ['udata', 'vdata'] ]

    # Wire up input RGB video:
    wires = []
    for i in range(3):
        j = 3 - i
        wires.append(rgb_v[i].wireup(rgb[j*BPP:(j-1)*BPP]))

    # Predefine YUV slices
    yuv_slices = (a[i][YUV_SLICE] for i in range(3) )
         
    wires += [
        yuv.wireup(
            concat(*yuv_slices)
        )   
    ]
    
    
    @pipeline(clk, None, ce = vin.dval, pass_in = vin, pass_out = vout)
    def yuv_pipe(ctx):
        """This contains the two-stage transformation for the RGB-YUV matrix"""
        yield [        
            y.set(py * rgb_v),
            u.set(pu.signed() * rgb_v),
            v.set(pv.signed() * rgb_v)        
        ]

        # Create sum expressions for readability:
        _y, _u, _v = (i.sum() for i in [y, u, v])
        
        yield [ 
            a[0].set(_y.signed()),
            a[1].set(_u),
            a[2].set(_v) 
        ]

    return locals()


### Testbench

In [8]:
from myirl.targets import VHDL
from myirl.test.common_test import run_ghdl

@block
def testbench_rgb2yuv():
    clk = ClkSignal(name = "pclk")
    yuv = Signal(intbv(0)[3*BPP:])
    vint, vout = [VideoPort() for _ in range(2)]
    
    yuv = Signal(intbv(0)[3*BPP:], name = 'yuv_data')
    rgb = Signal(intbv(0)[3*BPP:], name = 'rgb_data')

    testbench_rgb2yuv.insert(vout)
    
    inst = video_rgb_yuv(clk = clk,
                        vin = vint,
                        rgb = rgb,
                        param_matrix = MATRIX,
                        vout = vout,
                        yuv = yuv
    )
    
    osc = gen_osc(clk, CYCLE = 5)
    
    @sim.generator
    def stimulus():    
        
        # Feed a few color values:
        values = sim.Iterator([0x00ffff, 0x7f7f7f, 0x008300, 0x1a840a])
        
        yield [
            vint.dval.set(False), vint.fval.set(True), vint.lval.set(True),
            sim.wait(4 * [ clk.posedge, ] ), 
            vint.dval.set(True),
            sim.For(values)(
                sim.wait('1 ns'),
                rgb.set(values),
                sim.wait(2 * [clk.posedge]),
                sim.print_(yuv),
            ),

            sim.wait(3 * [ clk.posedge, ] ), 
            sim.assert_(vout.dval == True, "Video not valid"),
        ]

        for _ in range(3):
            yield [
                sim.print_(yuv),
                sim.wait(clk.posedge), 
            ]

        yield [
            sim.raise_(sim.StopSimulation)
        ]
        
    return locals()

def test():
    tb = testbench_rgb2yuv()
    files = tb.elab(VHDL, elab_all = True)
    run_ghdl(files, tb, debug = True, vcdfile="yuv.vcd")
    return tb

In [9]:
tb = test()

VHDL target: REGISTERING <class 'myirl.library.bulksignals.VideoPort'>
Creating sequential 'testbench_rgb2yuv/stimulus' 
 DEBUG: Writing 'video_rgb_yuv' to file /tmp/myirludebawxx/video_rgb_yuv.vhdl 
Finished _elab in 0.0031 secs
 DEBUG: Writing 'testbench_rgb2yuv' to file /tmp/myirludebawxx/testbench_rgb2yuv.vhdl 
Finished _elab in 0.1430 secs
==== COSIM stdout ====

==== COSIM stderr ====

==== COSIM stdout ====
analyze /tmp/myirludebawxx/module_defs.vhdl
analyze ../../myirl/targets/../test/vhdl/txt_util.vhdl
analyze ../../myirl/targets/libmyirl.vhdl
analyze /tmp/myirludebawxx/video_rgb_yuv.vhdl
analyze /tmp/myirludebawxx/testbench_rgb2yuv.vhdl
elaborate testbench_rgb2yuv

==== COSIM stderr ====

==== COSIM stdout ====
0x000000
0xB22B80
0x7E0000
0x4CD4C9
0x56D4D4
0x56D4D4
0x56D4D4
/tmp/myirludebawxx/testbench_rgb2yuv.vhdl:89:9:@175ns:(assertion failure): Stop Simulation
./testbench_rgb2yuv:error: assertion failed
in process .testbench_rgb2yuv(myirl).stimulus
./testbench_rgb2yuv:error

## Waveform trace

In [10]:
import wavedraw; import nbwavedrom
TB = tb.name;
cfg = {
    'tb.pclk' : None, 'tb.en' : None, 'tb.reset' : None,
    'tb.valid' : None, 'tb.data_out[4:0]' : None, 'tb.data_in[4:0]' : None, 
    'tb.inst_dummy_0.q[7:0]' : None, 'tb.inst_dummy_0.a[6:0]' : None,
}
waveform = wavedraw.vcd2wave("yuv.vcd", TB + '.pclk', None)
nbwavedrom.draw(waveform)

**Note** Bug in wave display. See correct trace: (Download) [yuv.vcd](yuv.vcd)

In [11]:
!cat -n {testbench_rgb2yuv.ctx.path_prefix}/video_rgb_yuv.vhdl 

     1	-- File generated from /usr/local/lib/python3.8/runpy.py
     2	-- (c) 2016-2021 section5.ch
     3	-- Modifications may be lost
     4	
     5	library IEEE;
     6	use IEEE.std_logic_1164.all;
     7	use IEEE.numeric_std.all;
     8	
     9	library work;
    10	
    11	use work.module_defs.all;
    12	use work.txt_util.all;
    13	use work.myirl_conversion.all;
    14	
    15	entity video_rgb_yuv is
    16	    port (
    17	        clk : in std_ulogic;
    18	        vin : in t_VideoPort;
    19	        rgb : in unsigned(23 downto 0);
    20	        vout : out t_VideoPort;
    21	        yuv : out unsigned(23 downto 0)
    22	    );
    23	end entity video_rgb_yuv;
    24	
    25	architecture MyIRL of video_rgb_yuv is
    26	    -- Local type declarations
    27	    -- Signal declarations
    28	    signal yuv_pipe_ce1 : std_ulogic;
    29	    signal yuv_pipe_ce2 : std_ulogic;
    30	    signal yuv_pipe_ce0 : std_ulogic;
    31	    type a_ydata is array (0 to 2) of unsigned(23 

## Verification exercise

Using numpy, we can run our samples through the floating point matrix as well:

In [12]:
v = numpy.matrix(mat_jpeg_rgb2yuv)
rgb = numpy.matrix([ (127, 127, 127), (0, 255, 255), (0, 0x83, 0)]).T

yuv = v * rgb

In [13]:
g = lambda x: "%02x" % (int(x) & 0xff)
f = numpy.vectorize(g)
f(yuv.T)

matrix([['7e', '00', '00'],
        ['b2', '2b', '81'],
        ['4c', 'd5', 'ca']], dtype='<U2')

Why will the results not completely match their Fractional multiplication counter parts? **Exercise**